In [2]:
## Importing necessary lib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
plt.style.use('dark_background')

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


In [2]:
tf.__version__

'2.8.0'

### S-1: Data Preprocessing :

In [3]:
## Importing Dataset
dataset = pd.read_csv("D:\Datasets\Churn_Modelling_Udm_A_Z.csv")

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.shape

(10000, 14)

In [6]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


#### Obs : 1. Dataset having 10000 row and 14 columns, among these column 1st three column i.e. rownumber , customerid, and surname is not important.
#### 2. Exited variable is Target here
#### 3. There is no missing value present in the dataset

In [8]:
x = dataset.iloc[:, 3:-1]    ## Feature var starting from 3rd colmn to last before
y = dataset.iloc[:, -1]      ## target variable

In [9]:
x.shape, y.shape

((10000, 10), (10000,))

In [10]:
x.Gender.value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [11]:
x.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

#### Obs : gender colmn has 2 unique value and Geography has 3 unique values which we have to encoded further

In [12]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [13]:
y.value_counts(normalize = True)*100

0    79.63
1    20.37
Name: Exited, dtype: float64

#### Obs : the datset taget variable is too much imbalanced i.e 79.63% of the customer is not exited but 20% of the customer is exited

### * Encoding of Categorical Feature 

In [14]:
x = dataset.iloc[:, 3:-1].values 
y = dataset.iloc[:, -1].values

In [15]:
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [16]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [17]:
from sklearn.preprocessing import LabelEncoder   ## for binary variable

le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [18]:
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

#### Obs : Female is encoded by 1 and male by 0

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')  ## 1 for colmun 1
x = np.array(ct.fit_transform(x))

In [20]:
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

#### Obs : Encod of France : 1,0,0 , Spain : 0,0,1 and Germany : 0,1,0

### * Data Splitting:

In [22]:
from sklearn.model_selection import train_test_split
np.random.seed(500)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
print('Shape of training dataset is :',x_train.shape)
print('')
print('Shape of test dataset is :',x_test.shape)

Shape of training dataset is : (8000, 12)

Shape of test dataset is : (2000, 12)


### * Feature Scaling

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## S-2 : Building ANN

In [24]:
## Determining the number of columns
input_dim = x_train.shape[1]

In [25]:
## Step- 2 : Creating First model
print('------------------ MODEL_1 ------------------')

# Creating / initialising an empty sequential model
model_1 = Sequential()

# Adding an input layer to the model or ANN or !st Hidden layer i.e Layer 1
model_1.add(Dense(6, input_dim = input_dim, activation = 'relu'))

# Adding 2nd hidden layer
model_1.add(Dense(6, activation = 'relu'))

# Adding an output layer to the model
model_1.add(Dense(1, activation = 'sigmoid'))   ## As we have to predict binary variable

# Train the ANN model
## Compile the model
model_1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

## Fit the model
model_1.fit(x_train, y_train, batch_size = 32, epochs = 100)

------------------ MODEL_1 ------------------
Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6113 - accuracy: 0.6775
Epoch 2/100
250/250 [==============================] - 0s 948us/step - loss: 0.4889 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 910us/step - loss: 0.4518 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 952us/step - loss: 0.4337 - accuracy: 0.8006
Epoch 5/100
250/250 [==============================] - 0s 990us/step - loss: 0.4246 - accuracy: 0.8083
Epoch 6/100
250/250 [==============================] - 0s 972us/step - loss: 0.4178 - accuracy: 0.8125
Epoch 7/100
250/250 [==============================] - 0s 958us/step - loss: 0.4105 - accuracy: 0.8188
Epoch 8/100
250/250 [==============================] - 0s 930us/step - loss: 0.4011 - accuracy: 0.8236
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3921 - accuracy: 0.8288
Epoch 10/100
250/250 [=========

In [26]:
## Evaluate the model
score_trn = model_1.evaluate(x_train, y_train)
print('Training Accuracy is : %.2f%%' %(100*score_trn[1]))
print('')

score_tst = model_1.evaluate(x_test, y_test)
print('Test Accuracy is : %.2f%%' %(100*score_tst[1]))
print('')

250/250 [==============================] - 0s 841us/step - loss: 0.3389 - accuracy: 0.8627
Training Accuracy is : 86.27%

63/63 [==============================] - 0s 1ms/step - loss: 0.3437 - accuracy: 0.8550
Test Accuracy is : 85.50%



#### Obs : Model is underfitting as model accuracy is not too good

In [27]:
print(model_1.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


#### Obs :
* Based on the following informations we have predicted should the customer say goodbye or not ?: Ans : No i.e Customer will not churned out or Exited
- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$ 60000
- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: \$ 50000
- Is this customer an Active Member: Yes


### * Making the Confusion Matrix

In [29]:
y_pred = model_1.predict(x_test)
y_pred = (y_pred > 0.5)

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)*100

[[1490  105]
 [ 185  220]]


85.5

## *** END ***